In [1]:
import configparser

import requests
# from parsel import Selector

import pandas as pd

In [2]:
subreddit      = "SanDiego"
subreddit_filt = "new"

In [3]:
# read in config file
config = configparser.ConfigParser()
config.read('reddit_config.ini')

['reddit_config.ini']

In [4]:
# Request access to reddit
auth = requests.auth.HTTPBasicAuth(config["script"]["personal_use_script"], config["script"]["secret"])

# set up log in creds
data = {'grant_type': 'password',
        'username': config["auth"]["username"],
        'password': config["auth"]["password"]
       }

headers = {'User-Agent': 'test_project'}

In [5]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [6]:
# res = requests.get("https://oauth.reddit.com/r/SanDiego/best",
#                    headers=headers, params={'limit':'5'})
res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{subreddit_filt}",
                   headers=headers)

In [7]:
reddit_res = res.json()

In [8]:
keys_interest = ['title', 'link_flair_text', 'selftext', 'url']
data_dict = {}
for i, d in enumerate(reddit_res['data']['children']):
    data_dict[i] = {k: d['data'][k] for k in keys_interest}
data_df = pd.DataFrame(data_dict).T
data_df

,title,link_flair_text,selftext,url
0,Caught sum pics of downtown san diego while wa...,Photo gallery,,https://www.reddit.com/gallery/yljsg1
1,Anyone know where I can sell gemstones in the ...,None,"Long story short, I recently inherited quite a...",https://www.reddit.com/r/sandiego/comments/ylj...
2,Joe Biden's massive security presence at the M...,Video,,https://streamable.com/un61qq
3,The Future Has Spoken!,Video,,https://v.redd.it/987220eqmtx91
4,Nicest extended stay hotel in Carlsbad????,None,Will be in the area for two or so months. Seem...,https://www.reddit.com/r/sandiego/comments/yli...
5,Question for the old heads - yogi cult living ...,None,I was showing an old friend a story about this...,https://www.reddit.com/r/sandiego/comments/ylh...
6,Any info on presidential route to Mira Costa c...,None,I’m thinking Miramar base to 15N to 78W? Not e...,https://www.reddit.com/r/sandiego/comments/ylg...
7,Civic theater weekday matinee parking?,None,I am going to see a Thursday matinee (2:30pm) ...,https://www.reddit.com/r/sandiego/comments/ylc...
8,What is it and what’s the purpose of it?,Photo,,https://i.redd.it/n83is717dsx91.jpg
9,Somewhere in San Diego - take a guess?,Photo,,https://i.redd.it/pdz19ha57sx91.jpg


In [9]:
reddit_res['data']['children'][0]['data'].keys()

dict_keys(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'is_gallery', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'media_metadata', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'ups', 'domain', 'media_embed', 'thumbnail_width', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'gallery_data', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'content_categories', 'is_self', 'subreddit_type', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'total_awards_received', 'all

In [10]:
reddit_res['data'].keys()

dict_keys(['after', 'dist', 'modhash', 'geo_filter', 'children', 'before'])